# Token Mixer
*alt name: Curio's GEnerator fOR Generating Embeddings (curio's george)* <br>
 by [aicrumb](https://twitter.com/aicrumb) for [ai_curio](https://twitter.com/ai_curio)

Launch a gradio demo that allows you to mix existing tokens to create a textual-inversion. Based on a currently unfinished text2img notebook from crumb.

In [1]:
#@title log into huggingface
import os
from google.colab import output

if not os.path.exists("installed.txt"):
    !pip install transformers gradio -q
    !pip install git+https://github.com/huggingface/diffusers -q
    !sudo apt-get install git-lfs
    !pip install OmegaConf -q
    !pip install wget -q
    !curl https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py > convert.py
    !cat "true" > installed.txt
    !curl https://gist.githubusercontent.com/aicrumb/3676fb3cdcd1d0c2ccca65729acea021/raw/983818e2f304b22c523947fcefd50f3c457e959c/test.py > inference.py
from huggingface_hub import notebook_login
output.clear()
notebook_login()

model_name = "CompVis/stable-diffusion-v1-4" #@param {type:"string"}

`mode` style will generate a picture in the style of x, mode will generate a picture of x <br>
`specific_concepts` concepts on huggingface (textual inversions) user/repo https://huggingface.co/sd-concepts-library <br>
`prompt` self explanatory, tokens to mix <br>
`negative` negative prompt <br>
`scheduler` self explanatory <br>
`steps` how long to run the image for <br>
`width_height` wow a lot of these are self explanatory <br>
`guidance_scale` how much to push the model to fit the prompt <br>
`seed` for reproducability <br>
`batch_size` i havent tested this >1 <br>
`disable_safety_checker` just in case

In [ ]:
#@title launch gradio demo

mode = "object" #@param ['object', 'style']
if mode=="style":
    global_prefix = "a picture in the style of "
else:
    global_prefix = "a picture of "

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipeline
from diffusers import LMSDiscreteScheduler, DDIMScheduler, KarrasVeScheduler, PNDMScheduler, DDPMScheduler
import torchvision.transforms.functional as f
from torch import nn
from IPython.display import display, Image, clear_output
from tqdm.auto import trange
from huggingface_hub import hf_hub_download
torch.backends.cudnn.benchmark = True
import gradio as gr
from PIL import Image

from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer
tokenizer = CLIPTokenizer.from_pretrained(
    model_name,
    subfolder="tokenizer"
)
text_encoder = CLIPTextModel.from_pretrained(
    model_name, subfolder="text_encoder"
).cpu()


lms = LMSDiscreteScheduler(
    beta_start=0.00085, 
    beta_end=0.012, 
    beta_schedule="scaled_linear"
)
ddim = DDIMScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear"
)
ddpm = DDPMScheduler( # tune these parameters for whatever reason, they do Not work
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear"
)
pndm = PNDMScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear"
)
import sys
print("loading model...")
pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    scheduler=lms,
)
pipe = pipe.to("cuda")
pipe.enable_attention_slicing()
import wget
import os
import requests
from huggingface_hub import HfApi
from huggingface_hub import hf_hub_download

def load_learned_embed_in_clip(learned_embeds_path, text_encoder, tokenizer, token=None):
        loaded_learned_embeds = torch.load(learned_embeds_path, map_location="cpu")
        trained_token = list(loaded_learned_embeds.keys())[0]
        embeds = loaded_learned_embeds[trained_token]
        dtype = text_encoder.get_input_embeddings().weight.dtype
        embeds.to(dtype)
        token = token if token is not None else trained_token
        num_added_tokens = tokenizer.add_tokens(token)
        i = 1
        text_encoder.resize_token_embeddings(len(tokenizer))
        token_id = tokenizer.convert_tokens_to_ids(token)
        text_encoder.get_input_embeddings().weight.data[token_id] = embeds
        return token

specific_concepts = [] #@param
models = []
for model in specific_concepts:
    model_content = {}
    model_content["id"] = model
    embeds = (model, "learned_embeds.bin")
    embeds_file = hf_hub_download(embeds[0], embeds[1])
    
    token_identifier = (model, "token_identifier.txt")
    tokens_file = hf_hub_download(token_identifier[0], token_identifier[1])
    # print(f"added {token_name}")
    token_name = open(tokens_file, "r").read()
    print("adding", token_name)

    learned_token = load_learned_embed_in_clip(embeds_file, pipe.text_encoder, pipe.tokenizer, token_name)
    model_content["token"] = learned_token
    models.append(model_content)

prompt = "heaven in hell dramatic painting starbucks caffeine frog" #@param {type:"string"}
tokens = tokenizer(prompt)['input_ids'][1:-1]
weights = [1]*len(tokens)
print("component parts:")
print([tokenizer.decode(i) for i in tokens])
print("weights:")
print(weights)

x = torch.cat([text_encoder.get_input_embeddings().weight[tokens[i]].unsqueeze(0) * weights[i] for i in range(len(tokens))],0).mean(0)
torch.save({"<my-style>": x.detach().cpu()}, "learned_embeds.bin")
load_learned_embed_in_clip("learned_embeds.bin", pipe.text_encoder, pipe.tokenizer, "<my-style>")
print("saved text inversion")

def generate():
        
    prompt = global_prefix + "<my-style>" 
    negative = "" #@param {type:"string"}

    scheduler = "lms" #@param ['ddim', 'lms', 'pndm']

    scheduler_map = {
        "lms": lms,
        "ddim": ddim, # someone please help get the right params for the ddim/ddpm samplers
        "ddpm": ddpm, # under maintanence
        "pndm": pndm,
        "default": pipe.scheduler # going to be useful later for clip guided sampling
    }
    pipe.scheduler = scheduler_map[scheduler]

    steps = 50 #@param
    # steps = steps - 9 # dude i have NO CLUE LOL
    width_height = [512, 512] #@param 
    width, height = width_height
    guidance_scale = 7.5 #@param
    seed = 42 #@param
    if seed == -1:
        import random
        seed = random.randint(1, 100_000)
        print("Using random seed:", seed)
    batch_size = 1 #@param
    batches = 1
    negative_prompt = None if len(negative)<=1 else negative
    disable_safety_checker = True #@param {type:'boolean'}
    use_pipe = pipe
    use_init = False
    with autocast("cuda"):
        if disable_safety_checker:
            # sometimes it's *really* bad and flags innocent fine images
            class DummyOutput(nn.Module):
                def __init__(self, pixel_values):
                    super().__init__()
                    self.hacking_into_the_mainframe = True
                    self.pixel_values = nn.Parameter(pixel_values)
                def forward(self):
                    return self.pixel_values

            class DummyFeatureExtractor:
                def __init__(self):
                    self.mainframe = "HAAACCKKEDD"
                def __call__(self, x, **extra):
                    pixel_values = f.to_tensor(x[0])
                    return DummyOutput(pixel_values)

            class DummySafetyChecker:
                def __init__(self):
                    self.hacker_code = "injected"
                def __call__(self, images, clip_input):
                    return images, False

            use_pipe.feature_extractor = DummyFeatureExtractor()
            use_pipe.safety_checker = DummySafetyChecker()
            import PIL.Image as PImage

            def image_grid(imgs, rows, cols):
                assert len(imgs) == rows*cols

                w, h = imgs[0].size
                grid = PImage.new('RGB', size=(cols*w, rows*h))
                grid_w, grid_h = grid.size
                
                for i, img in enumerate(imgs):
                    grid.paste(img, box=(i%cols*w, i//cols*h))
                return grid
        for i in trange(batches):
            generator = torch.cuda.manual_seed(seed+i)
            image = use_pipe(prompt, num_images_per_prompt=batch_size, negative_prompt=negative_prompt, width=width, height=height, guidance_scale=guidance_scale, generator=generator, num_inference_steps=steps).images  
            filename = "output/" + str(i).rjust(5, "0") + ".png"
            image = image_grid(image, 1, batch_size)
            image.save("output.png")
description = [tokenizer.decode(i) for i in tokens]
description = [str(i) + ": " + description[i] for i in range(len(description))]
def run(*weights):
    w = list(weights)
    tokens = tokenizer(prompt)['input_ids'][1:-1]
    x = torch.cat([text_encoder.get_input_embeddings().weight[tokens[i]].unsqueeze(0) * weights[i] for i in range(len(tokens))],0).sum(0)
    torch.save({"<my-style>": x.detach().cpu()*weights[-1]}, "learned_embeds.bin")
    load_learned_embed_in_clip("learned_embeds.bin", pipe.text_encoder, pipe.tokenizer, "<my-style>")
    # !python inference.py "CompVis/stable-diffusion-v1-4"
    generate()
    return Image.open("output.png")
demo = gr.Interface(
    fn=run,
    inputs=[gr.Slider(-1, 1, label=description[i], value=0) for i in range(len(weights))] + [gr.Slider(0, 2, label="normalize", value=1)],
    outputs=["image"],
    description='<hr>by <a href="https://twitter.com/aicrumb" target="_blank">aicrumb</a> for <a href="https://twitter.com/ai_curio" target="_blank">ai_curio</a><br>Launch a gradio demo that allows you to mix existing tokens to create a textual-inversion.',
    title="Token Mixer"
)

demo.launch(debug=True)  

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


loading model...


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


component parts:
['heaven', 'in', 'hell', 'dramatic', 'painting', 'starbucks', 'caffeine', 'frog']
weights:
[1, 1, 1, 1, 1, 1, 1, 1]
saved text inversion
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://18701.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
#@title download generated textual-inversion concept
#@markdown you can upload the files here to your own repo, or use them locally if you have a notebook that works with them
from google.colab import files
!mkdir concept

name = "<mixer-concept>" #@param {type:"string"}

x = torch.load("learned_embeds.bin")
torch.save({name: x['<my-style>'].detach().cpu()}, "learned_embeds.bin")

identifier = open("concept/token_identifier.txt", "w")
identifier.write(name)
identifier.close()

type_of_concept = open("concept/type_of_concept.txt", "w")
type_of_concept.write("object")
type_of_concept.close()

!cp learned_embeds.bin concept/learned_embeds.bin
!cp output.png concept/example.png

identifier = open("concept/README.md", "w")
identifier.write(f"""
---
tags:
- token-mixer
license: mit
---

{name} ![](example.png)""")
identifier.close()

!zip concept.zip concept/*

files.download('concept.zip')